In [5]:


import pandas as pd
import numpy as np
np.random.seed(1423)
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# import sys
# import matplotlib
# %matplotlib inline
# from matplotlib import pyplot as plt
# from sklearn.pipeline import Pipeline
# import theano
# import lasagne

In [6]:
def data_spread_IFT(filename):
    filename=filename[['TIME_PERIOD', 'OBS_VALUE']]
    filename['Year'] = filename['TIME_PERIOD'].str.split('-').str[0]

    filename = filename.rename(columns={'OBS_VALUE': 'UNRATE','TIME_PERIOD': 'DATE'})
    filename['DATE'] = pd.to_datetime(filename['DATE'])
    filename = filename[(filename['DATE'].dt.year >= 1994) & (filename['DATE'].dt.year <= 2021)]
    filename['Year'] = filename['DATE'].dt.year
    filename['Quarter'] = filename['DATE'].dt.quarter
    filename = filename.groupby(['Year', 'Quarter'])['UNRATE'].mean().reset_index()
    filename['DATE'] = filename['Year'].astype(str) + '-Q' + filename['Quarter'].astype(str)
    return  filename[['DATE', 'UNRATE']]


def data_Spread_Unrate(file_name):
    file_name = file_name.rename(columns={' Unemployment Rate (%)': 'UNRATE','Year': 'DATE'})
    file_name['DATE'] = file_name['DATE'].astype(int)
    file_name = file_name[(file_name['DATE'] >= 1994) & (file_name['DATE'] <= 2021)]
    file_name['Year'] = file_name['DATE']


    years = list(range(1994, 2022))
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']

    date_list = []
    for year in years:
        for quarter in quarters:
            date_list.append(str(year) + quarter)

    quarters_df = pd.DataFrame(date_list, columns=['DATE'])

    quarters_df['Year'] = quarters_df['DATE'].apply(lambda x: int(x[:4]))
    quarters_df['Quarter'] = quarters_df['DATE'].apply(lambda x: x[4:])

    file_name = pd.merge(quarters_df, file_name, on='Year', how='left')
    file_name['DATE'] = file_name['Year'].astype(str) + '-' + file_name['Quarter'].astype(str)
    return file_name[['DATE','UNRATE']]


def data_Spread_GDP(file_name):

    file_name = file_name.rename(columns={'Year': 'DATE'})
    file_name['DATE'] = file_name['DATE'].astype(int)
    file_name = file_name[(file_name['DATE'] >= 1994) & (file_name['DATE'] <= 2021)]
    file_name['Year'] = file_name['DATE']


    years = list(range(1994, 2022))
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']

    date_list = []
    for year in years:
        for quarter in quarters:
            date_list.append(str(year) + quarter)

    quarters_df = pd.DataFrame(date_list, columns=['DATE'])

    quarters_df['Year'] = quarters_df['DATE'].apply(lambda x: int(x[:4]))
    quarters_df['Quarter'] = quarters_df['DATE'].apply(lambda x: x[4:])

    file_name = pd.merge(quarters_df, file_name, on='Year', how='left')
    file_name['DATE'] = file_name['Year'].astype(str) + '-' + file_name['Quarter'].astype(str)
    return file_name[['DATE','GDP']]

def clean_duplicate(filename):
    start_time = '1994-Q1'
    end_time = '2021-Q4'
    filename = filename[(filename['TIME'] >= start_time) & (filename['TIME'] <= end_time)]
    filename = filename.reset_index(drop=True)
    return filename


In [7]:
une_USA=pd.read_csv('une_USA.csv')
une_USA['DATE'] = pd.to_datetime(une_USA['DATE'])
une_USA = une_USA[(une_USA['DATE'].dt.year >= 1994) & (une_USA['DATE'].dt.year <= 2021)]
une_USA['Year'] = une_USA['DATE'].dt.year
une_USA['Quarter'] = une_USA['DATE'].dt.quarter
une_USA = une_USA.groupby(['Year', 'Quarter'])['UNRATE'].mean().reset_index()
une_USA['DATE'] = une_USA['Year'].astype(str) + '-Q' + une_USA['Quarter'].astype(str)
une_USA = une_USA[['DATE', 'UNRATE']]


une_EU=pd.read_csv('une_EU.csv')

une_FRA = une_EU[une_EU['geo'] == 'FR']
une_DEU = une_EU[une_EU['geo'] == 'DE']
une_DNK = une_EU[une_EU['geo'] == 'DK']


une_FRA = data_spread_IFT(une_FRA)
une_DEU = data_spread_IFT(une_DEU)
une_DNK = data_spread_IFT(une_DNK)
# une_EU=une_EU[['TIME_PERIOD', 'OBS_VALUE']]
# une_EU['Year'] = une_EU['TIME_PERIOD'].str.split('-').str[0]
# une_EU = une_EU.rename(columns={'OBS_VALUE': 'UNRATE','TIME_PERIOD': 'DATE'})
# une_EU['DATE'] = pd.to_datetime(une_EU['DATE'])
# une_EU = une_EU[(une_EU['DATE'].dt.year >= 1994) & (une_EU['DATE'].dt.year <= 2021)]
# une_EU['Year'] = une_EU['DATE'].dt.year
# une_EU['Quarter'] = une_EU['DATE'].dt.quarter
# une_EU = une_EU.groupby(['Year', 'Quarter'])['UNRATE'].mean().reset_index()
# une_EU['DATE'] = une_EU['Year'].astype(str) + '-Q' + une_EU['Quarter'].astype(str)
# une_EU = une_EU[['DATE', 'UNRATE']]


une_CN=pd.read_csv('china-unemployment-rate.csv')
une_CN = une_CN.drop(range(0, 15))
une_CN = une_CN.drop(une_CN.columns[3], axis=1)
une_CN.columns = une_CN.iloc[0]
une_CN = une_CN.drop(une_CN.index[0])
une_CN.reset_index(drop=True, inplace=True)
une_CN['Year'] = une_CN['date'].str.split('/').str[0]
une_CN = une_CN.drop('date', axis=1)
une_CN = une_CN[[ 'Year', ' Unemployment Rate (%)']]
une_CN = data_Spread_Unrate(une_CN)


IFT=pd.read_csv('IFT_CNEUUSA.csv')

IFT_CN = IFT[IFT['LOCATION'] == 'CHN'][[ 'TIME', 'Value']]
IFT_CN = clean_duplicate(IFT_CN)
IFT_DNK = IFT[IFT['LOCATION'] == 'DNK'][[ 'TIME', 'Value']]
IFT_DNK = clean_duplicate(IFT_DNK)
IFT_FRA = IFT[IFT['LOCATION'] == 'FRA'][[ 'TIME', 'Value']]
IFT_FRA = clean_duplicate(IFT_FRA)
IFT_DEU = IFT[IFT['LOCATION'] == 'DEU'][[ 'TIME', 'Value']]
IFT_DEU = clean_duplicate(IFT_DEU)
IFT_USA = IFT[IFT['LOCATION'] == 'USA'][[ 'TIME', 'Value']]
IFT_USA = clean_duplicate(IFT_USA)




GDP = pd.read_csv('GDP2.csv')
GDP_CN = pd.DataFrame(GDP['40'].loc[4:])
GDP_CN = GDP_CN.rename(columns={GDP_CN.columns[0]: 'GDP'})
GDP_CN.insert(0, 'Year', range(1960, 2022))

GDP_DEU	 = pd.DataFrame(GDP['55'].loc[4:])
GDP_DEU	= GDP_DEU.rename(columns={GDP_DEU	.columns[0]: 'GDP'})
GDP_DEU	.insert(0, 'Year', range(1960, 2022))

GDP_FRA = pd.DataFrame(GDP['77'].loc[4:])
GDP_FRA= GDP_FRA.rename(columns={GDP_FRA.columns[0]: 'GDP'})
GDP_FRA.insert(0, 'Year', range(1960, 2022))

GDP_DNK = pd.DataFrame(GDP['58'].loc[4:])
GDP_DNK= GDP_DNK.rename(columns={GDP_DNK.columns[0]: 'GDP'})
GDP_DNK.insert(0, 'Year', range(1960, 2022))

GDP_USA = pd.DataFrame(GDP['251'].loc[4:])
GDP_USA= GDP_USA.rename(columns={GDP_USA.columns[0]: 'GDP'})
GDP_USA.insert(0, 'Year', range(1960, 2022))

GDP_CN = data_Spread_GDP(GDP_CN)
GDP_DEU = data_Spread_GDP(GDP_DEU)
GDP_FRA = data_Spread_GDP(GDP_FRA)
GDP_DNK = data_Spread_GDP(GDP_DNK)
GDP_USA = data_Spread_GDP(GDP_USA)

CN_data = pd.concat([GDP_CN, IFT_CN['Value'], une_CN['UNRATE']], axis=1)
USA_data = pd.concat([GDP_USA, IFT_USA['Value'], une_USA['UNRATE']], axis=1)
DEU_data = pd.concat([GDP_DEU, IFT_DEU['Value'], une_DEU['UNRATE']], axis=1)
DNK_data = pd.concat([GDP_DNK, IFT_DNK['Value'], une_DNK['UNRATE']], axis=1)
FRA_data = pd.concat([GDP_FRA, IFT_FRA['Value'], une_FRA['UNRATE']], axis=1)

CN_data['GDP'] = pd.to_numeric(CN_data['GDP'], errors='coerce')
CN_data['GDP'] = CN_data['GDP']  / 1e8


USA_data['GDP'] = pd.to_numeric(USA_data['GDP'], errors='coerce')
USA_data['GDP'] = USA_data['GDP']  / 1e8

DEU_data['GDP'] = pd.to_numeric(DEU_data['GDP'], errors='coerce')
DEU_data['GDP'] = DEU_data['GDP']  / 1e8

DNK_data['GDP'] = pd.to_numeric(DNK_data['GDP'], errors='coerce')
DNK_data['GDP'] = DNK_data['GDP']  / 1e8

FRA_data['GDP'] = pd.to_numeric(FRA_data['GDP'], errors='coerce')
FRA_data['GDP'] = FRA_data['GDP']  / 1e8


ift_cn = np.array(CN_data['Value']).reshape(-1,1)
une_cn = np.array(CN_data['UNRATE']).reshape(-1,1)
data_cn = np.hstack((ift_cn,une_cn))
gdp_cn = np.array(CN_data['GDP']).reshape(-1,1)
gdp_cn =  pd.DataFrame(gdp_cn)
data_cn =  pd.DataFrame(data_cn)

ift_usa = np.array(USA_data['Value']).reshape(-1,1)
une_usa = np.array(USA_data['UNRATE']).reshape(-1,1)
data_usa = np.hstack((ift_usa,une_usa))
gdp_usa = np.array(USA_data['GDP']).reshape(-1,1)
gdp_usa =  pd.DataFrame(gdp_usa)
data_usa =  pd.DataFrame(data_usa)

ift_dnk = np.array(DNK_data['Value']).reshape(-1,1)
une_dnk = np.array(DNK_data['UNRATE']).reshape(-1,1)
data_dnk = np.hstack((ift_dnk,une_dnk))
gdp_dnk = np.array(DNK_data['GDP']).reshape(-1,1)
gdp_dnk =  pd.DataFrame(gdp_dnk)
data_dnk =  pd.DataFrame(data_dnk)

ift_fra = np.array(FRA_data['Value']).reshape(-1,1)
une_fra = np.array(FRA_data['UNRATE']).reshape(-1,1)
data_fra = np.hstack((ift_fra,une_fra))
gdp_fra = np.array(FRA_data['GDP']).reshape(-1,1)
gdp_fra =  pd.DataFrame(gdp_fra)
data_fra =  pd.DataFrame(data_fra)

ift_deu = np.array(DEU_data['Value']).reshape(-1,1)
une_deu = np.array(DEU_data['UNRATE']).reshape(-1,1)
data_deu = np.hstack((ift_deu,une_deu))
gdp_deu = np.array(DEU_data['GDP']).reshape(-1,1)
gdp_deu =  pd.DataFrame(gdp_deu)
data_deu =  pd.DataFrame(data_deu)

merged_df = pd.concat([gdp_cn, gdp_usa], axis=0, ignore_index=True)
merged_df2 = pd.concat([merged_df, gdp_dnk], axis=0, ignore_index=True)
merged_df3 = pd.concat([merged_df2, gdp_fra], axis=0, ignore_index=True)
gdp_total = pd.concat([merged_df3, gdp_deu], axis=0, ignore_index=True)

merged_dfa = pd.concat([data_cn, data_usa], axis=0, ignore_index=True)
merged_dfb = pd.concat([merged_dfa, data_dnk], axis=0, ignore_index=True)
merged_dfc = pd.concat([merged_dfb, data_fra], axis=0, ignore_index=True)
data_total = pd.concat([merged_dfc, data_deu], axis=0, ignore_index=True)

C:\Users\20248\AppData\Local\Temp\ipykernel_284\3493353210.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filename['Year'] = filename['TIME_PERIOD'].str.split('-').str[0]
C:\Users\20248\AppData\Local\Temp\ipykernel_284\3493353210.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filename['Year'] = filename['TIME_PERIOD'].str.split('-').str[0]
C:\Users\20248\AppData\Local\Temp\ipykernel_284\3493353210.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [8]:
# CN_data['UNRATE'] = pd.to_numeric(CN_data['UNRATE'], errors='coerce')
# CN_data['Value'] = pd.to_numeric(CN_data['Value'], errors='coerce')

# CN_data['country'] = 'cn'
# DNK_data['country'] = 'dnk'
# USA_data['country'] = 'usa'
# DEU_data['country'] = 'deu'
# FRA_data['country'] = 'fra'


# ift_cn = np.array(CN_data['Value']).reshape(-1,1)
# une_cn = np.array(CN_data['UNRATE']).reshape(-1,1)
# data_cn = np.hstack((ift_cn,une_cn))
# gdp_cn = np.array(CN_data['GDP']).reshape(-1,1)
# gdp_cn =  pd.DataFrame(gdp_cn)
# data_cn =  pd.DataFrame(data_cn)

# ift_usa = np.array(USA_data['Value']).reshape(-1,1)
# une_usa = np.array(USA_data['UNRATE']).reshape(-1,1)
# data_usa = np.hstack((ift_usa,une_usa))
# gdp_usa = np.array(USA_data['GDP']).reshape(-1,1)
# gdp_usa =  pd.DataFrame(gdp_usa)
# data_usa =  pd.DataFrame(data_usa)

# ift_dnk = np.array(DNK_data['Value']).reshape(-1,1)
# une_dnk = np.array(DNK_data['UNRATE']).reshape(-1,1)
# data_dnk = np.hstack((ift_dnk,une_dnk))
# gdp_dnk = np.array(DNK_data['GDP']).reshape(-1,1)
# gdp_dnk =  pd.DataFrame(gdp_dnk)
# data_dnk =  pd.DataFrame(data_dnk)

# ift_fra = np.array(FRA_data['Value']).reshape(-1,1)
# une_fra = np.array(FRA_data['UNRATE']).reshape(-1,1)
# data_fra = np.hstack((ift_fra,une_fra))
# gdp_fra = np.array(FRA_data['GDP']).reshape(-1,1)
# gdp_fra =  pd.DataFrame(gdp_fra)
# data_fra =  pd.DataFrame(data_fra)

# ift_deu = np.array(DEU_data['Value']).reshape(-1,1)
# une_deu = np.array(DEU_data['UNRATE']).reshape(-1,1)
# data_deu = np.hstack((ift_deu,une_deu))
# gdp_deu = np.array(DEU_data['GDP']).reshape(-1,1)
# gdp_deu =  pd.DataFrame(gdp_deu)
# data_deu =  pd.DataFrame(data_deu)

# merged_df = pd.concat([gdp_cn, gdp_usa], axis=0, ignore_index=True)
# merged_df2 = pd.concat([merged_df, gdp_dnk], axis=0, ignore_index=True)
# merged_df3 = pd.concat([merged_df2, gdp_fra], axis=0, ignore_index=True)
# gdp_total = pd.concat([merged_df3, gdp_deu], axis=0, ignore_index=True)

# merged_df = pd.concat([data_cn, data_usa], axis=0, ignore_index=True)
# merged_df2 = pd.concat([merged_df, data_dnk], axis=0, ignore_index=True)
# merged_df3 = pd.concat([merged_df2, data_fra], axis=0, ignore_index=True)
# data_total = pd.concat([merged_df3, data_deu], axis=0, ignore_index=True)

In [9]:
# CN_data.to_csv('CN_data.csv')
def u(inflation, unrate,country):
    if country == 'cn':
        return  10261 * 0.762**(1 - float(unrate)) *(1 + float(inflation))
    if country == 'usa':
        return  65280 * 0.612**(1 - float(unrate)) *(1 + float(inflation))
    if country == 'fra':
        return  44670 * 0.645**(1 - float(unrate)) *(1 + float(inflation))
    if country == 'dnk':
        return  60170 * 0.768**(1 - float(unrate)) *(1 + float(inflation))
    if country == 'deu':
        return  50206 * 0.639**(1 - float(unrate)) *(1 + float(inflation))
        
Z_cn = np.array([u(ift_cn[i],une_cn[i],'cn') for i in range(0,112)]).reshape(-1,1)
Z_cn = pd.DataFrame(Z_cn)

Z_usa = np.array([u(ift_usa[i],une_usa[i],'usa') for i in range(0,112)]).reshape(-1,1)
Z_usa = pd.DataFrame(Z_usa)

Z_fra = np.array([u(ift_fra[i],une_fra[i],'fra') for i in range(0,112)]).reshape(-1,1)
Z_fra = pd.DataFrame(Z_fra)

Z_dnk = np.array([u(ift_dnk[i],une_dnk[i],'dnk') for i in range(0,112)]).reshape(-1,1)
Z_dnk = pd.DataFrame(Z_dnk)

Z_deu = np.array([u(ift_deu[i],une_deu[i],'deu') for i in range(0,112)]).reshape(-1,1)
Z_deu = pd.DataFrame(Z_deu)

merged_df11 = pd.concat([Z_cn, Z_usa], axis=0, ignore_index=True)
merged_df22 = pd.concat([merged_df11, Z_dnk], axis=0, ignore_index=True)
merged_df33 = pd.concat([merged_df22, Z_fra], axis=0, ignore_index=True)
Z_total = pd.concat([merged_df33, Z_deu], axis=0, ignore_index=True)
Z_total = np.array(Z_total)
# Z_total

In [10]:
#Regress X on Y using cubic regression. 
poly = PolynomialFeatures(degree=3)
Z_total_poly = poly.fit_transform(Z_total)

model = LinearRegression(fit_intercept=False)
model = model.fit(Z_total_poly, gdp_total)

X_estimator_prag = np.array([model.coef_[0][0] 
                            + model.coef_[0][1]*Z_total[i] 
                            + model.coef_[0][2]*Z_total[i]**2 
                            + model.coef_[0][3]*Z_total[i]**3 for i in range(0,112)])

In [11]:
# Cluster Xs.
N_CLASSES = 3
X_lbls_prag = KMeans(n_clusters=N_CLASSES, n_init=10,).fit_predict(X_estimator_prag.reshape(-1,1))

In [12]:
#Cluster Ys.
m = 112

Y_ftrs = np.zeros((une_cn.shape[0],m))
for i in range(0,m):
        helper_matrix = np.hstack((data_total,Z_total))
        helper_matrix = np.delete(helper_matrix,np.where(helper_matrix[:,1] != une_cn[i]),axis=0)
        Y_ftrs[i] = helper_matrix[:,2].mean()
print('Done. Clustering P(y | X_lbls).')
Y_lbls_prag = KMeans(n_clusters=N_CLASSES, n_init=10).fit_predict(Y_ftrs)

Done. Clustering P(y | X_lbls).


In [13]:
# We cite the code from Causal Feature Learning for Utility-Maximizing Agents to do the following computation
#Create labels showing how variables have been coarsened.

full_matrix=np.hstack((data_cn,X_lbls_prag.reshape(-1,1),Y_lbls_prag.reshape(-1,1)))

crs_C0 = np.delete(full_matrix,np.where(full_matrix[:,2]!=0),axis=0)
crs_C0 = crs_C0[:,0].reshape(-1,1)
crs_C0 = np.array([crs_C0[i] for i in range(0,len(crs_C0))])
crs_C0 = list(np.unique(crs_C0))
crs_C1 = np.delete(full_matrix,np.where(full_matrix[:,2]!=1),axis=0)
crs_C1 = crs_C1[:,0].reshape(-1,1)
crs_C1 = np.array([crs_C1[i] for i in range(0,len(crs_C1))])
crs_C1 = list(np.unique(crs_C1))
crs_C2 = np.delete(full_matrix,np.where(full_matrix[:,2]!=2),axis=0)
crs_C2 = crs_C2[:,0].reshape(-1,1)
crs_C2 = np.array([crs_C2[i] for i in range(0,len(crs_C2))])
crs_C2 = list(np.unique(crs_C2))


crs_E0 = np.delete(full_matrix,np.where(full_matrix[:,3]!=0),axis=0)
crs_E0 = crs_E0[:,1].reshape(-1,1)
crs_E0 = np.array([crs_E0[i] for i in range(0,len(crs_E0))])
crs_E0 = list(np.unique(crs_E0))
crs_E1 = np.delete(full_matrix,np.where(full_matrix[:,3]!=1),axis=0)
crs_E1 = crs_E1[:,1].reshape(-1,1)
crs_E1 = np.array([crs_E1[i] for i in range(0,len(crs_E1))])
crs_E1 = list(np.unique(crs_E1))
crs_E2 = np.delete(full_matrix,np.where(full_matrix[:,3]!=2),axis=0)
crs_E2 = crs_E2[:,1].reshape(-1,1)
crs_E2 = np.array([crs_E2[i] for i in range(0,len(crs_E2))])
crs_E2 = list(np.unique(crs_E2))

In [14]:
#Find the conditional probability table and print, with labels. Note the guide after the table showing which fine-grained variable values have been coarsened together. 
P_CE = np.array([np.bincount(Y_lbls_prag.astype(int)[X_lbls_prag==X_lbl],minlength=Y_lbls_prag.max()+1).astype(float) for X_lbl in np.sort(np.unique(X_lbls_prag))])
P_CE = P_CE/P_CE.sum()
P_E_given_C = np.around(P_CE/P_CE.sum(axis=1, keepdims=True),3)
column = np.array(['crs_C0','crs_C1','crs_C2']).reshape(-1,1)
header = np.array(['    ','crs_E0','crs_E1','crs_E2'])
P_E_given_C = np.hstack((column,P_E_given_C))
P_E_given_C = np.vstack((header,P_E_given_C))
print(P_E_given_C)
print('crs_C0=', crs_C0)
print('crs_C1=', crs_C1)
print('crs_C2=', crs_C2)

print('crs_E0=', crs_E0)
print('crs_E1=', crs_E1)
print('crs_E2=', crs_E2)


[['    ' 'crs_E0' 'crs_E1' 'crs_E2']
 ['crs_C0' '0.341' '0.0' '0.659']
 ['crs_C1' '0.0' '1.0' '0.0']
 ['crs_C2' '0.905' '0.095' '0.0']]
crs_C0= [-2.16709, -1.531023, -1.433831, -1.433728, -1.269247, -1.163939, -1.100066, -0.9913543, -0.8645086, -0.8327988, -0.7576911, -0.625702, -0.5931718, -0.5868979, -0.1337218, -0.1089662, 0.09692045, 0.1008965, 0.120048, 0.2640063, 0.2989442, 0.4696215, 0.5267693, 0.6094389, 0.66374, 0.7926206, 0.8069077, 0.9300591, 0.9993294, 1.000762, 1.147486, 1.268126, 1.307824, 1.357313, 1.379565, 1.40732, 1.414098, 1.437909, 1.450253, 1.508197, 1.567239, 1.592239, 1.602879, 1.628989, 1.658429, 1.679921, 1.711886, 1.782459, 1.79357, 1.822323, 1.829653, 1.89781, 1.905997, 1.932727, 1.958384, 2.071556, 2.102102, 2.103176, 2.10914, 2.126289, 2.128351, 2.138235, 2.160904, 2.173218, 2.24056, 2.285898, 2.390944, 2.463619, 2.580492, 2.62852, 2.667514, 2.718078, 2.72193, 2.733513, 2.745838, 2.747699, 2.763158, 2.771089, 2.864337, 2.894226, 2.933743, 3.089114, 3.264564